In [2]:
# !pip3 install tensorflow_hub

     |████████████████████████████████| 92kB 2.5MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.


In [9]:
import pandas as pd
import numpy as np
import spacy
from tqdm import tqdm
import re
import time
import pickle
pd.set_option('display.max_colwidth', 200)

In [18]:
# read data
train = pd.read_csv("train_2kmZucJ.csv")
test = pd.read_csv("test_oJQbWVk.csv")

train.shape, test.shape

((7920, 3), (1953, 2))

In [19]:
train['label'].value_counts(normalize = True)

0    0.744192
1    0.255808
Name: label, dtype: float64

In [20]:
train.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1MfQV #android #apps #beautiful #cute #health #igers #iphoneonly #iphonesia #iphone
1,2,0,Finally a transparant silicon case ^^ Thanks to my uncle :) #yay #Sony #Xperia #S #sonyexperias… http://instagram.com/p/YGEt5JC6JM/
2,3,0,We love this! Would you go? #talk #makememories #unplug #relax #iphone #smartphone #wifi #connect... http://fb.me/6N3LsUpCu
3,4,0,I'm wired I know I'm George I was made that way ;) #iphone #cute #daventry #home http://instagr.am/p/Li_5_ujS4k/
4,5,1,What amazing service! Apple won't even talk to me about a question I have unless I pay them $19.95 for their stupid support!


In [21]:
# remove URL's from train and test
train['clean_tweet'] = train['tweet'].apply(lambda x: re.sub(r'http\S+', '', x))

test['clean_tweet'] = test['tweet'].apply(lambda x: re.sub(r'http\S+', '', x))

In [22]:
# remove punctuation marks
punctuation = '!"#$%&()*+-/:;<=>?@[\\]^_`{|}~'

train['clean_tweet'] = train['clean_tweet'].apply(lambda x: ''.join(ch for ch in x if ch not in set(punctuation)))
test['clean_tweet'] = test['clean_tweet'].apply(lambda x: ''.join(ch for ch in x if ch not in set(punctuation)))

# convert text to lowercase
train['clean_tweet'] = train['clean_tweet'].str.lower()
test['clean_tweet'] = test['clean_tweet'].str.lower()

# remove numbers
train['clean_tweet'] = train['clean_tweet'].str.replace("[0-9]", " ")
test['clean_tweet'] = test['clean_tweet'].str.replace("[0-9]", " ")

# remove whitespaces
train['clean_tweet'] = train['clean_tweet'].apply(lambda x:' '.join(x.split()))
test['clean_tweet'] = test['clean_tweet'].apply(lambda x: ' '.join(x.split()))

In [23]:
# import spaCy's language model
nlp = spacy.load('en', disable=['parser', 'ner'])

# function to lemmatize text
def lemmatization(texts):
    output = []
    for i in texts:
        s = [token.lemma_ for token in nlp(i)]
        output.append(' '.join(s))
    return output

In [24]:
train['clean_tweet'] = lemmatization(train['clean_tweet'])
test['clean_tweet'] = lemmatization(test['clean_tweet'])

In [25]:
train.sample(10)

,id,label,tweet,clean_tweet
7404,7405,0,Just deleted all of my messages between Patrick and I accidentally iphone #mad,just delete all of -PRON- message between patrick and i accidentally iphone mad
6949,6950,0,Hey Guys! Look Nokia-8800-Sapphire-Arte-LUXUS-Handy-Phone http://zpr.io/6MP3Y #money #today #life #twitch #birthday #sale #android #windows #nokia #samsung #iphone #phone #smartphone #motorola #of...,"hey guy look nokia sapphirearteluxushandyphone money today life twitch birthday sale android window nokia samsung iphone phone smartphone motorola offer bestprice jul , pm"
5516,5517,0,Shot from the camera on a commercial shoot. . . . . . #video #photooftheday #beautiful #producer #live #fashion #videoproduction #follow #iphone #apple #actor #shotoniphone #thinkenvy #canon #cano...,shoot from the camera on a commercial shoot . . . . . . video photooftheday beautiful producer live fashion videoproduction follow iphone apple actor shotoniphone thinkenvy canon canonc c business...
1755,1756,0,@yosp I'm so happy when the PS4 will come out. Welcome 2 the future of gaming :) #PS4 #gaming #sony,yosp -PRON- be so happy when the ps will come out . welcome the future of gaming ps game sony
4833,4834,0,Gotta have a fashion case! #likeacharm #iphone #iphone5s #leopard #case #fashion #new http://instagram.com/p/lGCNXoN8oN/,get to have a fashion case likeacharm iphone iphone s leopard case fashion new
1891,1892,0,I’m Pretty Sure These Orangutans… http://dlvr.it/Lnj2St #news #photography #fashion #health #fail #tech #ipad #iphone #funny #lol,-PRON- be pretty sure these orangutan … news photography fashion health fail tech ipad iphone funny lol
6306,6307,0,10Pcs Exquisite Squishy Random Charm in http://ebay.to/2yI9MR7 #iPhone Straps #Toys Decor #free #shop #style #sales #today #giveawaypic.twitter.com/b8MSUXKiHR,pcs exquisite squishy random charm in iphone strap toy decor free shop style sale today giveawaypic.twitter.comb msuxkihr
53,54,0,Inbox: Letters to the Editor #news #photography #fashion #health #fail #tech #ipad #iphone #funny #lolpic.twitter.com/iK1RHUMoVz,inbox letter to the editor news photography fashion health fail tech ipad iphone funny lolpic.twitter.comik rhumovz
4541,4542,0,RT @umairsandhu #Woman uses #iPhone to get #Pregnant. http://ow.ly/1nApq2 #mm #Phone #Gadget #Technology #applle #apps,rt umairsandhu woman use iphone to get pregnant . mm phone gadget technology applle app
1004,1005,0,Like a geisha #kimono #geisha #jepang #pink #beauty #makeup #hairdo #red #sephora #iphone… https://instagram.com/p/1fZ3j_uG76/,like a geisha kimono geisha jepang pink beauty makeup hairdo red sephora iphone …


In [26]:
import tensorflow_hub as hub
import tensorflow as tf
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    # Legacy Python that doesn't verify HTTPS certificates by default
    pass
else:
    # Handle target environment that doesn't support HTTPS verification
    ssl._create_default_https_context = _create_unverified_https_context


elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

In [27]:
# just a random sentence
x = ["Roasted ants are a popular snack in Columbia"]

# Extract ELMo features 
embeddings = elmo(x, signature="default", as_dict=True)["elmo"]

embeddings.shape

TensorShape([Dimension(1), Dimension(8), Dimension(1024)])

In [28]:
def elmo_vectors(x):
  embeddings = elmo(x.tolist(), signature="default", as_dict=True)["elmo"]

  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    # return average of ELMo features
    return sess.run(tf.reduce_mean(embeddings,1))

In [29]:
list_train = [train[i:i+100] for i in range(0,train.shape[0],100)]
list_test = [test[i:i+100] for i in range(0,test.shape[0],100)]

In [ ]:
# Extract ELMo embeddings
elmo_train = [elmo_vectors(x['clean_tweet']) for x in list_train]
elmo_test = [elmo_vectors(x['clean_tweet']) for x in list_test]

In [ ]:
elmo_train_new = np.concatenate(elmo_train, axis = 0)
elmo_test_new = np.concatenate(elmo_test, axis = 0)

In [ ]:
# save elmo_train_new
pickle_out = open("elmo_train_03032019.pickle","wb")
pickle.dump(elmo_train_new, pickle_out)
pickle_out.close()

# save elmo_test_new
pickle_out = open("elmo_test_03032019.pickle","wb")
pickle.dump(elmo_test_new, pickle_out)
pickle_out.close()

In [ ]:
# load elmo_train_new
pickle_in = open("elmo_train_03032019.pickle", "rb")
elmo_train_new = pickle.load(pickle_in)

# load elmo_train_new
pickle_in = open("elmo_test_03032019.pickle", "rb")
elmo_test_new = pickle.load(pickle_in)

In [ ]:
from sklearn.model_selection import train_test_split

xtrain, xvalid, ytrain, yvalid = train_test_split(elmo_train_new, 
                                                  train['label'],  
                                                  random_state=42, 
                                                  test_size=0.2)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

lreg = LogisticRegression()
lreg.fit(xtrain, ytrain)

In [ ]:
preds_valid = lreg.predict(xvalid)

In [ ]:
f1_score(yvalid, preds_valid)

In [ ]:
# make predictions on test set
preds_test = lreg.predict(elmo_test_new)

In [ ]:
# prepare submission dataframe
sub = pd.DataFrame({'id':test['id'], 'label':preds_test})

# write predictions to a CSV file
sub.to_csv("sub_lreg.csv", index=False)